In [43]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence


Parameters

In [44]:
batch_size = 16
vocab_size = 5000
embedding_size = 32

Tokenize text

In [45]:
train_df = pd.read_csv('train.csv')
with open('train.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['text']))

spm.SentencePieceTrainer.Train('--input=train.txt --model_prefix=train --vocab_size={0} --model_type=unigram --max_sentence_length=9999'.format(vocab_size))
vocab_list = pd.read_csv('train.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list

sp = spm.SentencePieceProcessor()
sp.load('train.model')

True

Define x_train, y_train tensor

In [46]:
data_train = [0 for _ in range(len(train_df))]
for i in range(len(train_df)):
    data_train[i] = torch.FloatTensor(sp.encode_as_ids(train_df['text'][i]))

x_train = torch.transpose(pad_sequence(data_train), 0, 1)
y_train = torch.FloatTensor(train_df['target'])

x_train_maxlen = x_train.shape[-1]

Define Dataset

In [47]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x_data = x
        self.y_data = y
    
    def __len__(self):
        return len(self.y_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x, y

train_set = CustomDataset(x_train, y_train)

trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

Define Positional Embedding

In [48]:
class PositionalEmbedding(nn.Module):
    def __init__(self, max_len, vocab_size, embedding_dim):
        super(PositionalEmbedding, self).__init__()
        self.token_emb = nn.Embedding(vocab_size, embedding_dim)
        self.pos_emb = nn.Embedding(max_len, embedding_dim)
    
    def forward(self, x):
        max_len = x.shape[-1]
        positions = torch.range(start=0, end=max_len, step=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

Define Multi Head Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_dim, )

In [49]:
class Net(nn.Module):
    def __init__(self):
        self.embedding = PositionalEmbedding(x_train_maxlen, vocab_size, embedding_size)
        

    def forward(self, x):
        pass